In [4]:
from osdatahub import FeaturesAPI, Extent, NGD
import geojson
import pandas as pd
import geopandas as gpd
import shapely.wkt
import shapely.geometry
import os

from key import ngd_key

pd.set_option("display.max_rows", 100)

data_dir = "../data/geo_analysis/os_licensing/"
os.makedirs(data_dir, exist_ok=True)

In [5]:
def nrow(df):
    return print(f"No. of records in df: {len(df):,}")

## Data import

In [6]:
# LPA boundary data from planning.data.gov

LAD_boundary_df = pd.read_csv("https://files.planning.data.gov.uk/dataset/local-authority-district.csv", 
                                  usecols = ["reference", "name", "geometry"])

# LAD_boundary_df.columns = ["geometry", "name", "LPACD"]


# load geometry and create GDF
LAD_boundary_df['geometry'] = LAD_boundary_df['geometry'].apply(shapely.wkt.loads)
LAD_boundary_gdf = gpd.GeoDataFrame(LAD_boundary_df, geometry='geometry')

# Transform to ESPG:27700 for more interpretable area units
LAD_boundary_gdf.set_crs(epsg=4326, inplace=True)
LAD_boundary_gdf.to_crs(epsg=27700, inplace=True)

nrow(LAD_boundary_gdf)
LAD_boundary_gdf.head()


No. of records in df: 344


,geometry,name,reference
0,"MULTIPOLYGON (((450154.627 525938.188, 450164....",Hartlepool,E06000001
1,"MULTIPOLYGON (((446854.689 517192.726, 446858....",Middlesbrough,E06000002
2,"MULTIPOLYGON (((451747.383 520561.094, 451698....",Redcar and Cleveland,E06000003
3,"MULTIPOLYGON (((447177.708 517811.773, 447198....",Stockton-on-Tees,E06000004
4,"MULTIPOLYGON (((423496.594 524724.326, 423475....",Darlington,E06000005


In [7]:
# get camden listed building data direct from endpoint
cmd_df = pd.read_csv("https://opendata.camden.gov.uk/api/views/uu3n-zgbj/rows.csv?accessType=DOWNLOAD")

# load geometry and create GDF
cmd_df['geometry'] = cmd_df['geometry'].apply(shapely.wkt.loads)
cmd_gdf = gpd.GeoDataFrame(cmd_df[["reference", "geometry"]], geometry='geometry')

cmd_gdf["geogcd"] = "E09000007"

# Transform to ESPG:27700 for more interpretable area units
cmd_gdf.set_crs(epsg=4326, inplace=True)
cmd_gdf.to_crs(epsg=27700, inplace=True)

nrow(cmd_gdf)
cmd_gdf.head()

No. of records in df: 1,961


,reference,geometry,geogcd
0,LB1859,"POLYGON ((529794.083 183544.904, 529791.773 18...",E09000007
1,LB1481,"POLYGON ((525383.827 186281.856, 525383.774 18...",E09000007
2,LB1872,"POLYGON ((526706.983 184142.479, 526706.278 18...",E09000007
3,LB1531,"POLYGON ((529534.796 181777.495, 529525.359 18...",E09000007
4,LB1532,"POLYGON ((529598.623 181849.781, 529599.345 18...",E09000007


In [8]:
nrow(cmd_gdf)

No. of records in df: 1,961


## API test

In [9]:
# create a simplified geometry of the LAD polygon to use in API query
# this should mean downloading less unused data than just using a bounding box
# geometry needs to be simplified so query doesn't exceed max length, and buffered too so as to not clip out areas

area_code = "E09000007"
buffer_dist = 250
simp_tolerance = 250

area_geom = LAD_boundary_gdf[LAD_boundary_gdf["reference"] == area_code]["geometry"]
area_geom_simple = area_geom.simplify(simp_tolerance).buffer(buffer_dist).to_crs(4326).explode()
poly_str = shapely.wkt.dumps(area_geom_simple.values[0], rounding_precision=2)

len(poly_str)

/var/folders/0p/chrk2fd55l76zwfn62dphwvw0000gn/T/ipykernel_61324/2673653809.py:10: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  area_geom_simple = area_geom.simplify(simp_tolerance).buffer(buffer_dist).to_crs(4326).explode()


1947

In [10]:
coords = ((529000,181000), (530000,181000), (530000,182000), (529000,182000), (529000,181000))
bbox_poly = shapely.geometry.Polygon(coords)

map_lad = area_geom.explore(
    color = "blue"
)

area_geom_simple.explore(
    m = map_lad,
    color = "red"
)

In [12]:
collection = "bld-fts-buildingpart-1"
ngd = NGD(ngd_key, collection)


cql = f"INTERSECTS(geometry, {poly_str})"

results = ngd.query(
    max_results=10, 
    cql_filter = cql,
    crs = 27700, 
    offset = 0
)

test_gdf = gpd.GeoDataFrame.from_features(results["features"])
test_gdf.set_crs(epsg=27700, inplace=True)

nrow(test_gdf)

No. of records in df: 10


In [13]:
test_gdf.explore()

## Full API query

In [14]:
# API details
collection = "bld-fts-buildingpart-1"
ngd = NGD(ngd_key, collection)

# bounding box query
cql = f"INTERSECTS(geometry, {poly_str})"
# cql = f"INTERSECTS(geometry, {shapely.wkt.dumps(bbox_gdf.to_crs(27700).buffer(200)[0], rounding_precision=4)})"

fields = ["geometry", "osid", "versiondate"]

# controls
limit = 80000
interval = 100
fail_limit = 10

# data storage & counter
api_results = []
fail_counter = 0

for count, offset in enumerate(range(0, limit, interval)):

    if fail_counter > fail_limit:
        break
    
    print(f"attempt number {count}")
    try:
        results = ngd.query(cql_filter = cql, crs = 27700, max_results = interval, offset = offset)
        print("query success")

        results_gdf = gpd.GeoDataFrame.from_features(results["features"])
        results_gdf.set_crs(epsg=27700, inplace=True)

        api_results.append(results_gdf[fields])

    except:
        print("query fail")
        fail_counter += 1


    # api_results.append(results_gdf)

    # if there isn't a next page in the response then break
    if not any(d["rel"] == "next" for d in results["links"]):
        break
        

os_gdf = pd.concat(api_results)
nrow(os_gdf)

attempt number 0
query success
attempt number 1
query success
attempt number 2
query success
attempt number 3
query success
attempt number 4
query success
attempt number 5
query success
attempt number 6
query success
attempt number 7
query success
attempt number 8
query success
attempt number 9
query success
attempt number 10
query success
attempt number 11
query success
attempt number 12
query success
attempt number 13
query success
attempt number 14
query success
attempt number 15
query success
attempt number 16
query success
attempt number 17
query success
attempt number 18
query success
attempt number 19
query success
attempt number 20
query success
attempt number 21
query success
attempt number 22
query success
attempt number 23
query success
attempt number 24
query success
attempt number 25
query success
attempt number 26
query success
attempt number 27
query success
attempt number 28
query success
attempt number 29
query success
attempt number 30
query success
attempt number 31


In [45]:
# save or read in API results
# os_gdf.to_file(os.path.join(data_dir, "os_bld-fts-building-1_camden-full.gpkg"))
# os_gdf = gpd.read_file("os_bld-fts-building-1_camden-full.gpkg")

In [25]:
# filter the OS data to the LAD area
# os_clip_gdf = os_gdf.loc[
#     os_gdf.within(area_geom.iloc[0])
#     ].copy()

# os_clip_gdf.to_file(os.path.join(data_dir, "os_bld-fts-building-1_camden-extent.gpkg"))
os_clip_gdf = gpd.read_file(os.path.join(data_dir, "os_bld-fts-building-1_camden-extent.gpkg"))

In [26]:
os_clip_gdf.head()

,osid,versiondate,geometry
0,000100ea-e322-419a-82d8-d2be8b3a2834,2022-09-18,"POLYGON ((528963.600 183555.700, 528962.650 18..."
1,000178ea-1761-4443-a6d6-81b1b19d6ffc,2022-09-24,"POLYGON ((529377.400 184677.550, 529376.300 18..."
2,0001e4ed-d5eb-44bb-980a-a05db5ea0b4b,2022-09-18,"POLYGON ((530217.900 182087.100, 530226.300 18..."
3,00038807-93f3-47b6-b007-654a0f58b4ed,2022-09-18,"POLYGON ((528292.450 187096.550, 528294.900 18..."
4,000404cb-f48f-4208-bcf7-0a5c4f3681d1,2022-08-26,"POLYGON ((528981.400 185659.650, 528977.750 18..."


# Analysis

In [ ]:
# quick comparison before doing full intersection
# this is the total length of listed buildings perimiters as a % of the total length of OS building perimiters in the same bounding box
# a good estimate for what the max % of copying OS data might be if all listed building geoms are copied directly

pct_worst_case = sum(cmd_gdf.length) / sum(os_clip_gdf.length)
print("Worst-case match estimate: {:%}".format(round(pct_worst_case,3)))
print("(total listed building geometry perimeter length as a % of total OS building geometry perimiter length)")

Worst-case match estimate: 10.800000%
(total listed building geometry perimeter length as a % of total OS building geometry perimiter length)


In [28]:
# tidy names up to use in intersection and create area fields for each dataset
cmd_gdf["area"] = cmd_gdf["geometry"].area

cmd_gdf_join = cmd_gdf[["reference", "area", "geometry"]].copy()
cmd_gdf_join.columns = ["cmd_ref", "cmd_area", "geometry"]

os_clip_gdf["os_area"] = os_clip_gdf["geometry"].area
os_clip_gdf.rename(columns={"osid":"os_ref"}, inplace=True)

### Intersection

In [30]:
# first overlay to see which OS geoms match to listed building ones
cmd_os_join_gdf = gpd.overlay(
    cmd_gdf_join, 
    os_clip_gdf,
    how = "intersection", 
    keep_geom_type=False,
)

nrow(cmd_gdf_join)
nrow(os_clip_gdf)
nrow(cmd_os_join_gdf)

cmd_os_join_gdf["int_area"] = cmd_os_join_gdf["geometry"].area

# calculate intersection areas
cmd_os_join_gdf["cmd_int_pct"] = cmd_os_join_gdf["int_area"] / cmd_os_join_gdf["cmd_area"]
cmd_os_join_gdf["os_int_pct"] = cmd_os_join_gdf["int_area"] / cmd_os_join_gdf["os_area"]

# add a count field for the number of cmd > OS matches
cmd_os_join_gdf["os_match_count"] = cmd_os_join_gdf.groupby("cmd_ref")["cmd_ref"].transform("count")

No. of records in df: 1,961
No. of records in df: 51,055
No. of records in df: 8,162


### Direct matches

In [31]:
# CHECK FOR DIRECT MATCHES

threshold = 0.9

direct_matches = cmd_os_join_gdf[(cmd_os_join_gdf["cmd_int_pct"] >= threshold) & (cmd_os_join_gdf["os_int_pct"] > threshold)]
direct_match_pct = len(direct_matches) / len(os_clip_gdf)

print(f"no. of direct matches between listed building outlines and OS buildings: {len(direct_matches)}")
print(f"which equates to {round(direct_match_pct, 4):%} of all OS building geometries in area")
print("")

pct_best_case = sum(direct_matches.length) / sum(os_clip_gdf.length)
print("Best-case match estimate: {:%}".format(round(pct_best_case,3)))
print("(total matched listed building geometry perimeter length as a % of total OS building geometry perimiter length)")


map_os = os_clip_gdf[os_clip_gdf["os_ref"].isin(direct_matches["os_ref"])].explore(
    color = "blue",
    tooltip = False,
    tiles = "CartoDB positron"
)

cmd_gdf[cmd_gdf["reference"].isin(direct_matches["cmd_ref"])].explore(
    m = map_os,
    color = "red",
    # tooltip = False,
    style_kwds = {
        "fillOpacity" : "0"
        }
)

no. of direct matches between listed building outlines and OS buildings: 462
which equates to 0.900000% of all OS building geometries in area

Best-case match estimate: 1.700000%
(total matched listed building geometry perimeter length as a % of total OS building geometry perimiter length)


### Extending match limits through dissolving

In [32]:
# dissolve matching OS geometries

# lookup for cmd and os ids where matches are 1:1 or 1:many (where the many OS matches are each > 20%)
cmd_os_lookup = cmd_os_join_gdf[
    (cmd_os_join_gdf["os_match_count"] == 1) | ((cmd_os_join_gdf["os_match_count"] > 1) & (cmd_os_join_gdf["os_int_pct"] > 0.2))
    ][["cmd_ref", "os_ref"]]

# inner join to lookup and then dissolve OS geoms grouped by the listed building ref
os_dissolved = os_clip_gdf.merge(
    cmd_os_lookup,
    how = "inner",
    on = "os_ref"
).dissolve(
    by = "cmd_ref"
)

In [42]:
# map dissolved version of OS geometries with listed buildings they match to 
# map_os = os_dissolved.explore(
#     color = "blue",
#     tooltip = False,
#     tiles = "CartoDB positron"
# )

# cmd_gdf.explore(
#     m = map_os,
#     color = "red",
#     # tooltip = False,
#     style_kwds = {
#         "fillOpacity" : "0"
#         }
# )

In [34]:
# tidy dissolved table and re-calculate area for new dissolved geometries
os_dissolved.reset_index(inplace = True)
os_dissolved["os_area"] = os_dissolved["geometry"].area

os_dissolved.head()

,cmd_ref,geometry,os_ref,versiondate,os_area
0,LB1000,"POLYGON ((528980.700 183419.600, 528980.950 18...",0343fed1-35c9-4317-847b-80ede710bbac,2022-08-26,56.315000
1,LB1001,"POLYGON ((528893.856 183429.075, 528892.262 18...",000df1d7-e2e5-4239-9fad-072da8318c61,2022-09-18,1732.205380
2,LB1002,"POLYGON ((528802.050 183546.750, 528803.150 18...",3297611f-46d9-4aa8-acef-1b6b27565809,2022-08-26,117.205750
3,LB1003,"POLYGON ((528791.151 183558.500, 528792.900 18...",6c0a7631-f87e-423d-bdaf-79f08969f852,2022-08-26,164.944887
4,LB1004,"POLYGON ((528762.700 183535.500, 528769.950 18...",4e71f146-97ed-4e08-ba34-33de7fe6e19a,2022-11-28,237.331250


In [35]:
# intersect listed buildings with dissolved OS geometries
cmd_join_diss = gpd.overlay(
    cmd_gdf_join, 
    os_dissolved.reset_index(),
    how = "intersection", 
    keep_geom_type=False,
)

# filter to just those which have matching references (i.e. the dissolved OS geometry is based on the listed building geom)

cmd_join_diss = cmd_join_diss[cmd_join_diss["cmd_ref_1"] == cmd_join_diss["cmd_ref_2"]]
cmd_join_diss["int_area"] = cmd_join_diss["geometry"].area

nrow(cmd_gdf_join)
nrow(cmd_join_diss)
cmd_join_diss.head()

No. of records in df: 1,961
No. of records in df: 1,518


,cmd_ref_1,cmd_area,index,cmd_ref_2,os_ref,versiondate,os_area,geometry,int_area
0,LB1859,251.340384,675,LB1859,63303935-2b4a-48c6-baba-a4eac149e642,2022-08-26,264.922069,"POLYGON ((529791.773 183548.406, 529794.067 18...",250.559855
1,LB1872,2711.157851,686,LB1872,71bc0a46-3317-4d0e-ba77-12d4e2a87d1c,2024-01-29,2321.299950,"POLYGON ((526706.550 184144.451, 526706.301 18...",2317.468171
2,LB1531,626.057066,476,LB1531,c5c77ead-90b6-4670-ae12-2822379019a0,2022-09-18,589.548559,"POLYGON ((529528.932 181785.850, 529531.210 18...",589.518904
3,LB1533,1180.225467,477,LB1533,135b76dd-12cc-471b-ba7e-203ad9c19e43,2023-06-03,1180.217500,"POLYGON ((530161.652 181082.295, 530159.700 18...",1178.524689
6,LB423,164.543118,980,LB423,0d9a602e-e89f-4fad-ba83-0c5d9337a8f8,2022-08-26,159.967500,"POLYGON ((530117.506 181081.231, 530117.508 18...",159.593200


In [36]:
# check some examples from table above to see how area breakdowns work

ref = "LB446"

map_os = os_dissolved[os_dissolved["cmd_ref"] == ref].explore(
    color = "blue",
    tooltip = False,
    tiles = "CartoDB positron"
)

cmd_gdf[cmd_gdf["reference"] == ref].explore(
    m = map_os,
    color = "red",
    # tooltip = False,
    style_kwds = {
        "fillOpacity" : "0.3"
        }
)

In [37]:
# not all camden listed buildings intersect so rather than using overlay table, left join to it from the original listed building table
cmd_match_areas = cmd_gdf_join[["cmd_ref", "cmd_area", "geometry"]].merge(
    cmd_join_diss[["cmd_ref_1", "os_area", "int_area"]],
    left_on = "cmd_ref",
    right_on = "cmd_ref_1",
    how = "left"
)

cmd_match_areas["cmd_int_pct"] = cmd_match_areas["int_area"] / cmd_match_areas["cmd_area"]
cmd_match_areas["os_int_pct"] = cmd_match_areas["int_area"] / cmd_match_areas["os_area"]

cmd_match_areas.head()


,cmd_ref,cmd_area,geometry,cmd_ref_1,os_area,int_area,cmd_int_pct,os_int_pct
0,LB1859,251.340384,"POLYGON ((529794.083 183544.904, 529791.773 18...",LB1859,264.922069,250.559855,0.996895,0.945787
1,LB1481,2.618381,"POLYGON ((525383.827 186281.856, 525383.774 18...",NaN,NaN,NaN,NaN,NaN
2,LB1872,2711.157851,"POLYGON ((526706.983 184142.479, 526706.278 18...",LB1872,2321.299950,2317.468171,0.854789,0.998349
3,LB1531,626.057066,"POLYGON ((529534.796 181777.495, 529525.359 18...",LB1531,589.548559,589.518904,0.941638,0.999950
4,LB1532,2.462515,"POLYGON ((529598.623 181849.781, 529599.345 18...",NaN,NaN,NaN,NaN,NaN


In [69]:
cmd_os_lookup.head()

,cmd_ref,os_ref
0,LB1859,63303935-2b4a-48c6-baba-a4eac149e642
4,LB1872,71bc0a46-3317-4d0e-ba77-12d4e2a87d1c
5,LB1872,832e39f3-5d60-416b-b515-146d9c599d83
7,LB1872,d319afa3-1865-4ddc-831a-98b6b3c83f61
10,LB1531,c5c77ead-90b6-4670-ae12-2822379019a0


In [38]:
# set threshold for combined overlap
threshold = 0.8

# table of listed buildings where the combined match to a dissolved OS geometry is over the threshold 
cmd_match_thresh = cmd_match_areas[(cmd_match_areas["cmd_int_pct"] >= threshold) & (cmd_match_areas["os_int_pct"] >= threshold)]

pct_dissolved_matches = len(cmd_match_thresh) / len(cmd_match_areas)


# table of all OS geoms which went into the dissolved geoms which have a match over threshold
# os_match_geoms = cmd_os_lookup[cmd_os_lookup["cmd_ref"].isin(cmd_match_thresh["cmd_ref"])].merge(
#     os_gdf, 
#     how = "inner", 
#     left_on = "os_ref", 
#     right_on="osid")

os_match_geoms = os_clip_gdf.merge(
    cmd_os_lookup[cmd_os_lookup["cmd_ref"].isin(cmd_match_thresh["cmd_ref"])][["os_ref"]].drop_duplicates(),
    how = "inner", 
    on="os_ref"
)

os_no_match_geoms = os_clip_gdf.merge(
    cmd_os_lookup[~cmd_os_lookup["cmd_ref"].isin(cmd_match_thresh["cmd_ref"])][["os_ref"]].drop_duplicates(),
    how = "inner", 
    on="os_ref"
)

pct_os_match_geoms = len(os_match_geoms) / len(os_clip_gdf)

print("{} out of {} listed  building geoms have a combined match over threshold to OS dissolved geoms".format(len(cmd_match_thresh), len(cmd_gdf_join)))
print("this equates to {:%} %".format(round(pct_dissolved_matches, 3)))

print("")
print("{} out of {} distinct OS building geoms are included in the dissolved geoms which match listed building outlines".format(len(os_match_geoms), len(os_clip_gdf)))
print("this equates to {:%} %".format(round(pct_os_match_geoms, 3)))

print("")
pct_middle_case = sum(cmd_match_thresh.length) / sum(os_clip_gdf.length)
print("Middle-case match estimate: {:%}".format(round(pct_middle_case,3)))
print("(total dissolve-matched listed building geometry perimeter length as a % of total OS building geometry perimiter length)")


1112 out of 1961 listed  building geoms have a combined match over threshold to OS dissolved geoms
this equates to 56.700000% %

4183 out of 51055 distinct OS building geoms are included in the dissolved geoms which match listed building outlines
this equates to 8.200000% %

Middle-case match estimate: 6.400000%
(total dissolve-matched listed building geometry perimeter length as a % of total OS building geometry perimiter length)


In [45]:
# map_os = os_no_match_geoms.explore(
#     color = "#68afff",  # blue for geoms not matched
#     tooltip = False,
#     tiles = "CartoDB positron",
# )

# map_os2 = os_match_geoms.explore(
#     m = map_os,
#     color = "#53ffa2",  # green for geoms matched
#     tooltip = False
# )

# cmd_gdf_join.explore(
#     m = map_os2,
#     color = "red",
#     # tooltip = False,
#     style_kwds = {
#         "fillOpacity" : "0"
#         }
# )

In [44]:
# map_os = os_dissolved[~os_dissolved["cmd_ref"].isin(cmd_match_thresh["cmd_ref"])].explore(
#     color = "#68afff",  # blue for geoms not matched
#     tooltip = False,
#     tiles = "CartoDB positron"
# )

# map_os2 = os_dissolved[os_dissolved["cmd_ref"].isin(cmd_match_thresh["cmd_ref"].drop_duplicates())].explore(
#     m = map_os,
#     color = "#53ffa2",  # green for geoms matched
#     tooltip = False
# )

# cmd_gdf_join.explore(
#     m = map_os2,
#     color = "red",
#     # tooltip = False,
#     style_kwds = {
#         "fillOpacity" : "0"
#         }
# )